# RH Experiments — Visualization Notebook

This notebook reproduces and visualizes experiments for thin-band suppression coefficients in the NB/BD framework.

**Files used (if present):**
- `eoff_experiments.csv`
- `multi_eoff_experiments.csv`
- `family_search_alpha_beta.csv`
- `largeN_eoff_results.csv`

If files are missing, the notebook will run small experiments to create them.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import mobius
from pathlib import Path

DATA = Path('/mnt/data')
plt.rcParams['figure.figsize'] = (6,4)

In [ ]:
def gaussian(x, lam):
    return np.exp(-0.5 * (x/lam)**2)

def coefficients(N, alpha=0.75, beta=0.25):
    a = np.zeros(N+1, dtype=np.float64)
    lnN = np.log(max(2, N))
    for n in range(1, N+1):
        mu = mobius(n)
        if mu == 0:
            continue
        ln = np.log(n)
        c = (-alpha * gaussian(ln, 0.5*lnN)
             + (1+beta) * gaussian(ln, lnN)
             - alpha * gaussian(ln, 2.0*lnN))
        a[n] = mu/np.sqrt(n) * c
    return a

def eoff(a):
    N = len(a)-1
    total = 0.0
    for m in range(1, N+1):
        am = abs(a[m])
        if am == 0.0: 
            continue
        for n in range(1, N+1):
            if n == m: 
                continue
            total += am * abs(a[n]) * np.exp(-0.5*abs(np.log(m/n)))
    return total

def run_experiment(N_list=(200,500,1000), alpha=0.75, beta=0.25, out_csv=DATA/'multi_results.ipynb_generated.csv'):
    rows = []
    for N in N_list:
        a = coefficients(N, alpha, beta)
        sum_sq = float(np.sum(a**2))
        E = float(eoff(a))
        ratio = E/sum_sq if sum_sq>0 else float('inf')
        rows.append((N, alpha, beta, sum_sq, E, ratio))
        print(f'N={N}: ratio={ratio:.3f}')
    df = pd.DataFrame(rows, columns=['N','alpha','beta','sum_sq','E_off','ratio'])
    df.to_csv(out_csv, index=False)
    return df

In [ ]:
# Load CSVs if present, else create minimal ones
paths = {
    'eoff': DATA/'eoff_experiments.csv',
    'multi': DATA/'multi_eoff_experiments.csv',
    'family': DATA/'family_search_alpha_beta.csv',
    'large': DATA/'largeN_eoff_results.csv'
}
dfs = {}
for k,p in paths.items():
    if p.exists():
        dfs[k] = pd.read_csv(p)
        print(f'Loaded {k} from {p}')
    else:
        dfs[k] = None
        print(f'Missing {k}: {p}')

In [ ]:
# If multi/large missing, run a small experiment to generate a minimal dataset
if dfs['multi'] is None:
    dfs['multi'] = run_experiment(N_list=(200,500), alpha=0.75, beta=0.25, out_csv=DATA/'multi_eoff_experiments.ipynb_generated.csv')
if dfs['large'] is None:
    dfs['large'] = run_experiment(N_list=(800,1000), alpha=0.75, beta=0.25, out_csv=DATA/'largeN_eoff_results.ipynb_generated.csv')

## Plot: Large-N ratios
A simple line plot of ratios vs N for available configurations.

In [ ]:
# Plot ratios vs N using 'large' dataset if present
if dfs['large'] is not None:
    df = dfs['large']
    if 'config' in df.columns:
        for name, group in df.groupby('config'):
            plt.figure()
            plt.plot(group['N'], group['ratio'], marker='o')
            plt.xlabel('N')
            plt.ylabel('E_off / sum|a_n|^2')
            plt.title(f'Large-N ratio — {name}')
            plt.grid(True)
            plt.show()
    else:
        plt.figure()
        plt.plot(df['N'], df['ratio'], marker='o')
        plt.xlabel('N'); plt.ylabel('E_off / sum|a_n|^2')
        plt.title('Large-N ratio')
        plt.grid(True); plt.show()

## Parameter scan (optional)
Scan a small grid of $(\alpha,\beta)$ to visualize ratio changes (slow for large $N$).

In [ ]:
# Small alpha,beta scan for N=200 (kept small to be fast)
def scan_alpha_beta(N=200, alphas=(0.5,0.75,1.0), betas=(0.0,0.25,0.5)):
    rows = []
    for a in alphas:
        for b in betas:
            A = coefficients(N, a, b)
            S = float(np.sum(A**2))
            E = float(eoff(A))
            r = E/S if S>0 else float('inf')
            rows.append((N,a,b,S,E,r))
            print(f'N={N}, alpha={a}, beta={b}, ratio={r:.3f}')
    return pd.DataFrame(rows, columns=['N','alpha','beta','sum_sq','E_off','ratio'])

scan_df = scan_alpha_beta(N=200)
scan_df.head()

In [ ]:
# Scatter plot of ratio vs parameters (encoded on axes)
plt.figure()
for _,row in scan_df.iterrows():
    plt.scatter(row['alpha'] + 0.001*row['beta'], row['ratio'], marker='o')
plt.xlabel('alpha (with beta jitter)')
plt.ylabel('ratio')
plt.title('Parameter scan — ratios (N=200)')
plt.grid(True)
plt.show()